<a href="https://colab.research.google.com/github/Biswamitra4599/mri_setup/blob/main/Bart_sense_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#USING BART


## Importing System Libraries & Mounting Drive


In [42]:
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image
import h5py
import glob
import shutil


In [38]:
#Mounting Google Drive
# !rm -rf drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [43]:
# Remove an empty directory
mri_path="/content/drive/MyDrive/MRI/"
cfl_path=mri_path+"CFL/"
buffer_path=mri_path+"BUFFER/"
h5_path=mri_path+"H5/"
test_path=mri_path+"TEST/"

# Check if the directory exists
if os.path.exists(buffer_path) and os.path.isdir(buffer_path):
    try:
        # Force delete the directory and all its contents
        shutil.rmtree(buffer_path)
        print(f"Directory '{buffer_path}' and all its contents have been deleted.")
    except Exception as e:
        print(f"Error while deleting the directory: {e}")
else:
    print(f"The directory '{buffer_path}' does not exist.")


# try:
#     os.rmdir(buffer_path)
#     print(f"Directory '{buffer_path}' removed successfully.")
# except OSError as e:
#     print(f"Error: {e.strerror}")

Directory '/content/drive/MyDrive/MRI/BUFFER/' and all its contents have been deleted.


In [44]:
# Managing the Paths to avoid any Error
if not os.path.exists(mri_path):
  os.makedirs(mri_path)
  os.makedirs(cfl_path)
  os.makedirs(buffer_path)
  os.makedirs(h5_path)
  os.makedirs(test_path)
  print(f"Path Created!!! Put the '.h5' files in the specific place: {h5_path}")
else:
  os.makedirs(buffer_path)
  print("Path Already Exists!!")


Path Already Exists!!


## Fast MRI Setup

In [45]:
!pip install fastmri

In [46]:
import fastmri
from fastmri.data import transforms as T
from fastmri.data.subsample import RandomMaskFunc

## Sense Reconstruction
by: Mahesh (SIR)

### Setting up the GPU and the appt environment

In [ ]:
%%bash

# Use CUDA 10.1 when on Tesla K80

# Estimate GPU Type
GPU_NAME=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader)

echo "GPU Type:"
echo $GPU_NAME

if [ "Tesla K80" = "$GPU_NAME" ];
then
    echo "GPU type Tesla K80 does not support CUDA 11. Set CUDA to version 10.1."

    # Change default CUDA to version 10.1
    cd /usr/local
    rm cuda
    ln -s cuda-10.1 cuda
    echo "done!!!"
else
    echo "Current GPU supports default CUDA-11."
    echo "No further actions are necessary."
fi

echo "GPU Information:"
nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
nvcc --version

### BART INSTALLATION

1.  Dependencies for BART,
2.  Installation of BART,
3.  Environment Setup.







In [ ]:
%%bash

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Clone Bart
[ -d /content/bart ] && rm -r /content/bart
git clone https://github.com/mrirecon/bart/ bart &> /dev/null
echo "Dependencies Done!!"
pwd

In [ ]:
%%bash
pwd
BRANCH=ISMRM21

cd bart

pwd
# Switch to desired branch of the BART project
git checkout -q $BRANCH

pwd

# Define specifications
COMPILE_SPECS=" PARALLEL=1
                CUDA=1
                CUDA_BASE=/usr/local/cuda
                CUDA_LIB=lib64
                OPENBLAS=1
                BLAS_THREADSAFE=1"

printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

make &> /dev/null


/content
/content/bart
/content/bart


In [ ]:
# Define environment variables for BART and OpenMP
os.environ['TOOLBOX_PATH'] = "/content/bart"
os.environ['OMP_NUM_THREADS']="4"
# Add the BARTs toolbox to the PATH variable
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append("/content/bart/python")
print(sys.path)

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/bart/python']


In [47]:

!echo "# The BART used in this notebook:"
!cd bart/
!which bart
!echo "# BART version: "
!bart version
!bart


# The BART used in this notebook:
/content/bart/bart
# BART version: 
v0.7.00-88-g9d66c09
BART. Available commands are:
avg         bench       bin         bitmask     cabs        caldir      
calmat      carg        casorati    cc          ccapply     cdf97       
circshift   conj        conv        conway      copy        cpyphs      
creal       crop        delta       ecalib      ecaltwo     epg         
estdelay    estdims     estshift    estvar      extract     fakeksp     
fft         fftmod      fftrot      fftshift    filter      flatten     
flip        fmac        homodyne    index       invert      itsense     
join        looklocker  lrmatrix    mandelbrot  mip         moba        
mobafit     nlinv       noise       normalize   nrmse       nufft       
ones        pattern     phantom     pics        pocsense    poisson     
pol2mask    poly        repmat      reshape     resize      rmfreq      
rof         roistat     rss         rtnlinv     sake        saxpy       
scal

## Moving into the Problem

### Remaining Libs

In [48]:
# More python libraries
import cfl

### Setup Visualization Helper
https://github.com/mrirecon/bart/blob/master/python/cfl.py

https://colab.research.google.com/github/mrirecon/bart-workshop/blob/master/mri_together_2021/bart_intro_examples.ipynb#scrollTo=__DLJ4XLDwXx


In [49]:
def plot_map(dataname, colorbar, vmin, vmax, cbar_label):

    # Import data
    data = np.abs(cfl.readcfl(dataname).squeeze())

    # Import colorbar type
    colorbartype =  colorbar

    # Set zero to a black color for a masking effect
    my_cmap = cm.get_cmap(colorbartype, 256)
    my_cmap.set_bad('black')

    data = np.ma.masked_equal(data, 0)

    fig = plt.figure(figsize=(15,15))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(data, interpolation='nearest', cmap=my_cmap, vmin=vmin, vmax=vmax)

    # Style settings
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label(cbar_label)
    cbar.ax.tick_params()

    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.xaxis.set_ticks_position('none')
    ax1.yaxis.set_ticks_position('none')
    ax1.set_axis_off()

    plt.show()

### Function definations

In [50]:
#fastmri_to_bart changes the dimentions appropriately to the cfl files
def fastmri_to_bart(data_kspace, slice_no):
    if slice_no == None:
        slice_kspace = data_kspace
    else:
        slice_kspace = data_kspace[slice_no,:,:,:]
    slice_kspace_format = np.transpose(slice_kspace, axes=[1,2,0])
    slice_kspace_format2=np.expand_dims(slice_kspace_format, axis=2)
    return slice_kspace_format2

# slice_wrapper = fastmri_to_bart(volume_kspace,1)

# Get a list of all .h5 files in the directory
def get_file_list():
  h5_files = glob.glob(os.path.join(h5_path, "*.h5"))
  ll=[]

  for h5_file in h5_files:
    temp=h5_file.replace(h5_path,"")
    temp=temp.replace(".h5","")
    ll.append(temp)
  return ll

#Getting User Input on Image
def get_user_input(ll):
  for index,tt in enumerate(ll):
    print(index,":\t", tt)
  print("\nEnter the index of the file name from the above list: ")
  ins = input()
  return int(ins)

#Extracting Volume K Space for specified Image.
def get_volume_kspace(h5_file_name):
  hf= h5py.File(h5_file_name)
  volume_kspace = hf['kspace'][()]
  return volume_kspace


### H5 file Reading

In [53]:
# H5 File Reading
h5_file_list=get_file_list()
ins=get_user_input(h5_file_list)
h5_file_name=h5_path+h5_file_list[int(ins)]+".h5"

#Getting the Volume K Space Data
volume_kspace = get_volume_kspace(h5_file_name)
print(volume_kspace.shape)

0 :	 file_brain_AXFLAIR_200_6002452
1 :	 file_brain_AXFLAIR_200_6002467
2 :	 file_brain_AXT1POST_201_6002673
3 :	 file_brain_AXT2_210_6001944

Enter the index of the file name from the above list: 
0
(16, 20, 640, 320)


In [59]:
# Extracting Slice from volume K Space
slice_kspace = volume_kspace[1]
print("Slice K-Space size: ",slice_kspace.shape)

slice_kspace_mid=int(slice_kspace.shape[2]/2)
print(slice_kspace_mid)
temp_ref=slice_kspace[:,:,slice_kspace_mid-16:slice_kspace_mid+17]
print(temp_ref.shape)

reference=fastmri_to_bart(slice_kspace,None)
print("Slice K-Space size(in .cfl format): ",reference.shape)

# #Converting to Tensors
# slice_kspace2 = T.to_tensor(slice_kspace)
# print(slice_kspace2.dtype)

Slice K-Space size:  (20, 640, 320)
160
(20, 640, 33)


In [ ]:
# Extracting Slice from volume K Space
slice_kspace = volume_kspace[1]
print("Slice K-Space size: ",slice_kspace.shape)

reference=fastmri_to_bart(slice_kspace,None)
print("Slice K-Space size(in .cfl format): ",reference.shape)

#Converting to Tensors
slice_kspace2 = T.to_tensor(slice_kspace)
print(slice_kspace2.dtype)

In [ ]:
# Create the mask function object
cntr_frac=0.04
acc_fact=2
mask_func = RandomMaskFunc(center_fractions=[cntr_frac], accelerations=[acc_fact])

# Apply the mask to the sliced k-space
masked_kspace = T.apply_mask(slice_kspace2, mask_func)

# Masked Space Combination
# print(type(masked_kspace))
# print(masked_kspace[0].dtype)
# print(masked_kspace[0].shape)

masked_kspace_npy = masked_kspace[0].numpy()
# print(masked_kspace_npy.shape)
masked_kspace_C_npy = masked_kspace_npy[:,:,:,0] + 1j* masked_kspace_npy[:,:,:,1]
# print(masked_kspace_C_npy.shape)

k_fastmri_ds = fastmri_to_bart(masked_kspace_C_npy, None)
# print(k_fastmri_ds.shape)


In [ ]:
# print(mask_func)

In [ ]:
sampled_path=buffer_path+"downsampled_fastmri"
reference_path=buffer_path+"fullspace_fastmri"
# print(sampled_path)
# print(reference_path)
cfl.writecfl(sampled_path,k_fastmri_ds )
cfl.writecfl(reference_path,reference )


### Visualizing the Data

#### Full Spaced Image

In [ ]:
%%bash
bart ecalib -r24 -m1 /content/drive/MyDrive/MRI/BUFFER/fullspace_fastmri /content/drive/MyDrive/MRI/CFL/coils_l
FFT_FLAG=$(bart bitmask 0 1)
COIL_FLAG=$(bart bitmask 3)
bart fft -i -u $FFT_FLAG /content/drive/MyDrive/MRI/BUFFER/fullspace_fastmri /content/drive/MyDrive/MRI/CFL/coil_image
bart fmac -C -s$COIL_FLAG /content/drive/MyDrive/MRI/CFL/coil_image /content/drive/MyDrive/MRI/CFL/coils_l /content/drive/MyDrive/MRI/CFL/image_fs

In [ ]:
#Visualizing actual Image(reference Image)
ds_image_path=cfl_path + "image_fs"
pics_ds = cfl.readcfl(ds_image_path)
print(np.shape(pics_ds))
vmax=0.5*np.max(np.abs(pics_ds))
plt.imshow(np.abs(pics_ds), cmap="gray", vmax=vmax)
plt.show()

#### Down Sampled Image

In [ ]:
%%bash
bart ecalib -r24 -m1 /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/CFL/coils_l
FFT_FLAG=$(bart bitmask 0 1)
COIL_FLAG=$(bart bitmask 3)
bart fft -i -u $FFT_FLAG /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/CFL/coil_image
bart fmac -C -s$COIL_FLAG /content/drive/MyDrive/MRI/CFL/coil_image /content/drive/MyDrive/MRI/CFL/coils_l /content/drive/MyDrive/MRI/CFL/image_ds

In [ ]:
fs_image_path=cfl_path + "image_ds"
pics_ref = cfl.readcfl(fs_image_path)

print(np.shape(pics_ref))
vmax=0.5*np.max(np.abs(pics_ref))
plt.imshow(np.abs(pics_ref), cmap="gray", vmax=vmax)
plt.show()

### Reconstruction Process

In [ ]:
#BART Image Reconstruction
!bart ecalib /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/BUFFER/sensitivities
!bart pics -l1 -r0.001 /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/BUFFER/sensitivities /content/drive/MyDrive/MRI/CFL/image_out_fmri

In [ ]:
#BART Reconstructed Image Visualization
reconstructed_image_path=cfl_path + "image_out_fmri"
pics_reco = cfl.readcfl(reconstructed_image_path)
vmax=0.5*np.max(np.abs(pics_reco))
pics_reco_sq = np.squeeze(pics_reco)
print(np.shape(pics_reco_sq))

# fig, axes = plt.subplots(figsize=(20,6), nrows=1, ncols=3, sharex=True, sharey=True)

plt.imshow(np.abs(pics_reco_sq[:,:,0]), cmap="gray", vmax=vmax)
plt.show()


In [ ]:
#Difference Map Plotting
diff_map = np.abs(pics_reco_sq[:,:,0])/np.max(np.abs(pics_reco_sq[:,:,0])) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
plt.show()

In [ ]:
diff_map = np.abs(pics_ds)/np.max(np.abs(pics_ds)) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
plt.show()

## Removing the BUFFER

In [ ]:
# Check if the directory exists
if os.path.exists(buffer_path) and os.path.isdir(buffer_path):
    try:
        # Force delete the directory and all its contents
        shutil.rmtree(buffer_path)
        print(f"Directory '{buffer_path}' and all its contents have been deleted.")
    except Exception as e:
        print(f"Error while deleting the directory: {e}")
else:
    print(f"The directory '{buffer_path}' does not exist.")

